# End-to-End Machine Learning Project 

## Training the model: example 1

In [ ]:
from joblib import dump
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
iris = datasets.load_iris(return_X_y=True)
X = iris[0]
y = iris[1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
clf_pipeline = [('scaling', MinMaxScaler()), 
                ('clf', RandomForestClassifier(random_state=42))]

pipeline = Pipeline(clf_pipeline)

pipeline.fit(X_train, y_train)

In [ ]:
dump(pipeline, 'data/iris_dt_v1.joblib')

## Creating the API: example 1

https://fastapi.tiangolo.com/

File: `app.py`

Run: `uvicorn app:app --reload`

Docs: `http://localhost:8000/docs`

## Training the model: example 2

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
import joblib
import gzip

In [ ]:
# Preselected feature
selected_features = [
    'concavity_mean',
    'concave_points_mean',
    'perimeter_se',
    'area_se',
    'texture_worst',
    'area_worst'
]

In [ ]:
# Load the dataset
data = pd.read_csv('data/breast_cancer.csv')

# Preprocess dataset
data = data.set_index('id')
data['diagnosis'] = data['diagnosis'].replace(['B', 'M'], [0, 1])  # Encode y, B -> 0 , M -> 1

y = data.pop('diagnosis')
X = data
X = X[selected_features.copy()]

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create an ensemble of 3 models
estimators = []
estimators.append(('logistic', LogisticRegression()))
estimators.append(('cart', DecisionTreeClassifier()))
estimators.append(('svm', SVC()))

In [ ]:
# Create the Ensemble Model
ensemble = VotingClassifier(estimators)

# Make preprocess Pipeline
pipe = Pipeline([
    ('imputer', SimpleImputer()),  # Missing value Imputer
    ('scaler', MinMaxScaler(feature_range=(0, 1))),  # Min Max Scaler
    ('model', ensemble)  # Ensemble Model
])

In [ ]:
# Train the model
pipe.fit(X_train, y_train)

In [ ]:
# Test Accuracy
print("Accuracy: %s%%" % str(round(pipe.score(X_test, y_test), 3) * 100))

In [ ]:
# Export model
joblib.dump(pipe, gzip.open('data/model_binary.dat.gz', "wb"))

## Creating the API: example 2

https://fastapi.tiangolo.com/

File: `app.py`

Run: `uvicorn cancer:app --reload`

Docs: `http://localhost:8000/docs`

    {
      "concavity_mean": 0.3001,
      "concave_points_mean": 0.1471,
      "perimeter_se": 8.589,
      "area_se": 153.4,
      "texture_worst": 17.33,
      "area_worst": 2019.0
    }